In [3]:
!pip install transformers peft bitsandbytes trl accelerate -q

In [4]:
import transformers
import peft
import bitsandbytes
import trl
import accelerate

print("transformers:", transformers.__version__)
print("peft:", peft.__version__)
print("bitsandbytes:", bitsandbytes.__version__)
print("trl:", trl.__version__)
print("accelerate:", accelerate.__version__)

transformers: 5.0.0
peft: 0.18.1
bitsandbytes: 0.49.2
trl: 0.28.0
accelerate: 1.12.0


In [5]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={
    "train": "/content/train.jsonl",
    "val": "/content/val.jsonl"
})

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1023
    })
    val: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 116
    })
})


In [6]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
print("Model selected:", model_name)

Model selected: TinyLlama/TinyLlama-1.1B-Chat-v1.0


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

print("Model loaded!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded!


In [8]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


In [15]:
def format_sample(sample):
    if sample['input'].strip():
        return f"Instruction:\n{sample['instruction']}\n\n Input:\n{sample['input']}\n\n Output:\n{sample['output']}"
    else:
        return f"Instruction:\n{sample['instruction']}\n\n Output:\n{sample['output']}"

# Test it
print(format_sample(dataset['train'][0]))

### Instruction:
Formulate an SQL query for finding the students who have scored over 80% in the test.

### Output:
SELECT * FROM Students
WHERE test_score > 80


In [18]:
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['val'],
    processing_class=tokenizer,
    args=SFTConfig(
        output_dir="adapters",
        num_train_epochs=3,
        per_device_train_batch_size=4,
        learning_rate=2e-4,
        logging_steps=10,
        save_strategy="epoch",
        fp16=False,
    ),
    formatting_func=format_sample,
)

trainer.train()

Step,Training Loss
10,1.310492
20,0.960611
30,0.973215
40,0.813090
50,0.755911
60,0.882057
70,0.861371
80,0.851068
90,0.794943
100,0.799825


TrainOutput(global_step=768, training_loss=0.7644613521794478, metrics={'train_runtime': 435.9476, 'train_samples_per_second': 7.04, 'train_steps_per_second': 1.762, 'total_flos': 3505458601881600.0, 'train_loss': 0.7644613521794478})

In [19]:
model.save_pretrained("adapters")
tokenizer.save_pretrained("adapters")
print("Adapter weights saved!")

Adapter weights saved!
